В этом блокноте рассматривается распознавание эмоций на корпусе CEDR с помощью векторных представлений ELMo и ансамбля классификаторов на основе разных алгоритмов. Корпус и подход описаны в статье ["Data-Driven Model for Emotion Detection in Russian Texts"](https://www.sciencedirect.com/science/article/pii/S1877050921013247).

Обученные модели доступны по ссылкам:
- [elmo_vec.pkl](https://canvas.instructure.com/courses/7745797/files/228148829?module_item_id=94185010)
- [joy_model.pkl](https://canvas.instructure.com/courses/7745797/files/228148847?module_item_id=94185018)
- [sad_model.pkl](https://canvas.instructure.com/courses/7745797/files/228148879?module_item_id=94185037)
- [surprise_model.pkl](https://canvas.instructure.com/courses/7745797/files/228148898?module_item_id=94185056)
- [fear_model.pkl](https://canvas.instructure.com/courses/7745797/files/228148913?module_item_id=94185063)
- [anger_model.pkl](https://canvas.instructure.com/courses/7745797/files/228148917?module_item_id=94185064)

Их необходимо скачать и поместить в одну папку.

Для запуска блокнота требуется версия питона 3.8.

In [1]:
import sys
print(sys.version)

3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]


### Установка библиотек

Использование моделей возможно только с определенными версиями библиотек, их необходимо установить.

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
!pip install TPOT==0.11.1 xgboost==0.90 numpy==1.21.2 scikit-learn==0.22.1 scipy==1.10.1 deap==1.3.3 pandas==2.0.3 joblib==1.3.1 tqdm==4.59 -q

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

huggingface-hub 0.17.1 requires packaging>=20.9, but you'll have packaging 20.4 which is incompatible.
datasets 2.14.5 requires tqdm>=4.62.1, but you'll have tqdm 4.59.0 which is incompatible.


### Загрузка данных

Набор данных доступен в библиотеке Datasets от Hugging Face.

In [4]:
!pip install datasets -q

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

huggingface-hub 0.17.1 requires packaging>=20.9, but you'll have packaging 20.4 which is incompatible.


Для загрузки набора данных используется метод load_dataset. Для классификации понадобится тестовая выборка основного корпуса, а также словарь, где указано соответствие числовых меток и классов эмоций.

In [5]:
from datasets import load_dataset

test_data = load_dataset('cedr', split='test')

labels2emotion = {0: "joy", 1: "sadness", 2: "surprise", 3: "fear", 4: "anger"}

### Загрузка моделей

Для загрузки модели используется модуль pickle. Формат pickle позволяет хранить любые объекты Python. Одной из его главных функций является сохранение модели машинного обучения.

In [6]:
import pickle

def load_model(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

В переменной path_to_data нужно указать путь к папке, в которой сохранены модели.

In [7]:
path_to_data = 'C:/models'

Можно загрузить все модели и посмотреть, какие алгоритмы использовались для их обучения.

In [8]:
joy_model = load_model(f'{path_to_data}/joy_model.pkl')
joy_model

LinearSVC(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.01,
          verbose=0)

In [9]:
sad_model = load_model(f'{path_to_data}/sad_model.pkl')
sad_model

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
surprise_model = load_model(f'{path_to_data}/surprise_model.pkl')
surprise_model

Pipeline(memory=None,
         steps=[('robustscaler',
                 RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                              with_centering=True, with_scaling=True)),
                ('stackingestimator',
                 StackingEstimator(estimator=LogisticRegression(C=0.01,
                                                                class_weight=None,
                                                                dual=False,
                                                                fit_intercept=True,
                                                                intercept_scaling=1,
                                                                l1_ratio=None,
                                                                max_iter=100,
                                                                multi_class='auto',
                                                                n_jobs=None,
                                               

In [11]:
fear_model = load_model(f'{path_to_data}/fear_model.pkl')
fear_model

Pipeline(memory=None,
         steps=[('pca',
                 PCA(copy=True, iterated_power=5, n_components=None,
                     random_state=None, svd_solver='randomized', tol=0.0,
                     whiten=False)),
                ('sgdclassifier',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.01,
                               fit_intercept=True, l1_ratio=0.25,
                               learning_rate='constant', loss='squared_hinge',
                               max_iter=1000, n_iter_no_change=5, n_jobs=None,
                               penalty='elasticnet', power_t=0.5,
                               random_state=None, shuffle=True, tol=0.001,
                               validation_fraction=0.1, verbose=0,
                               warm_start=False))],
         verbose=False)

In [12]:
anger_model = load_model(f'{path_to_data}/anger_model.pkl')
anger_model

LogisticRegression(C=20.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Файл с векторами ELMo представляет собой словарь с двумя ключами: train и test.

In [13]:
df = load_model(f'{path_to_data}/elmo_vec.pkl')
df.keys()

dict_keys(['train', 'test'])

Каждое значение словаря является списком, длина списка равна количеству объектов в соответствующей выборке.

In [14]:
len(df['train'])

7528

In [15]:
len(df['test'])

1882

Каждый элемент списка представляет собой словарь из векторов и эмоциональных меток.

In [16]:
df['train'][94]

{'vec': array([-0.2145    ,  0.10968809,  0.08183508, ..., -0.24737515,
        -0.05388971,  0.26529486]),
 'labels': ['sadness']}

In [17]:
df['test'][67]

{'vec': array([-0.31789889, -0.16927155,  0.37030656, ..., -0.2655307 ,
        -0.20555632,  0.28399004]),
 'labels': ['joy']}

Векторное представление и название класса можно получить по соответствующему ключу.

In [18]:
df['test'][95]['vec']

array([-0.21446268, -0.06942602,  0.29757945, ..., -0.20662831,
       -0.25054309, -0.04707903])

In [19]:
df['test'][95]['labels']

['surprise']

### Применение моделей для классификации

In [20]:
import numpy as np
from sklearn.metrics import f1_score, classification_report

# матрица для записи правильных и предсказанных ответов
# её размер — количество объектов х количество классов
all_true = np.zeros((1882,6))
all_pred = np.zeros((1882,6))

# будем осуществлять проход по всем предложениям 5 раз (по количеству эмоций)
for key, value in labels2emotion.items():
    if value == 'joy':
        model = joy_model
    elif value == 'sadness':
        model = sad_model
    elif value == 'surprise':
        model = surprise_model
    elif value == 'fear':
        model = fear_model
    elif value == 'anger':
        model = anger_model
        
    # векторы текстов и ответы тестовой выборки
    test_x, test_y = [], []
    
    # добавляем предложения в тестовую выборку
    for sample in df['test']:
        test_x.append(sample['vec'])
        # для каждого предложения проверяем наличие эмоции среди списка меток
        if value in sample['labels']:
            test_y.append(1)
        else:
            test_y.append(0)
    
    # записываем предсказания обученной модели
    pred_y = model.predict(np.array(test_x))
    
    # подсчитываем микро- и макроусредненную F-меру
    f_micro = f1_score(test_y, pred_y, average="micro")
    f_macro = f1_score(test_y, pred_y, average="macro")
    # выводим название класса и значения метрик
    print(f'Emotion "{value}":')
    print(f'mic.: {round(f_micro, 2)};\t mac.:{round(f_macro, 2)}\n')
    
    # записываем ответы для всех объектов в столбец с соответвующим индексом
    all_true[:, key] = np.array(test_y)
    all_pred[:, key] = pred_y

# ответы для нейтрального класса определяются по остаточному принципу
# если сумма всех меток эмоций равна нулю, то присваивается нейтральный класс
all_pred[:, 5] = (np.sum(all_pred[:,:], axis=1)==0).astype(int)
all_true[:, 5] = (np.sum(all_true[:,:], axis=1)==0).astype(int)
# считаем и выводим метрики
f_micro = f1_score(all_true[:, 5], all_pred[:, 5], average="micro")
f_macro = f1_score(all_true[:, 5], all_pred[:, 5], average="macro")
print(f'No emotion')
print(f'F-score\t micro: {round(f_micro, 2)};\tmacro:{round(f_macro, 2)}\n')

# выводим отчет о классификации
target_names = ["joy", "sadness", "surprise", "fear", "anger", "no emotion"]
print(classification_report(all_true, all_pred, target_names=target_names))

Emotion "joy":
mic.: 0.92;	 mac.:0.87

Emotion "sadness":
mic.: 0.92;	 mac.:0.86

Emotion "surprise":
mic.: 0.93;	 mac.:0.76

Emotion "fear":
mic.: 0.93;	 mac.:0.73

Emotion "anger":
mic.: 0.9;	 mac.:0.62

No emotion
F-score	 micro: 0.77;	macro:0.77

              precision    recall  f1-score   support

         joy       0.85      0.72      0.78       353
     sadness       0.85      0.72      0.78       379
    surprise       0.59      0.54      0.57       170
        fear       0.58      0.44      0.50       141
       anger       0.27      0.31      0.29       125
  no emotion       0.65      0.86      0.74       734

   micro avg       0.68      0.71      0.69      1902
   macro avg       0.63      0.60      0.61      1902
weighted avg       0.69      0.71      0.69      1902
 samples avg       0.69      0.71      0.70      1902

